In [ ]:
# need this package for the pipeline script
!pip install category_encoders

[cuml.accel] Failed to check cudaDevAttrConcurrentManagedAccess with error 35


In [2]:
import cuml
from cuml.ensemble import RandomForestRegressor as cuRF_Regressor
from cuml.ensemble import RandomForestClassifier as cuRF_Classifier
# Install with desired log level before other imports
cuml.accel.install(log_level="debug")

[cuml.accel] Failed to check cudaDevAttrConcurrentManagedAccess with error 35
[cuml.accel] Could not enable managed memory on this platform.
[cuml.accel] Accelerator installed.


ls: cannot access '/content/drive/My Drive/CIS 5200 Final Project/code': No such file or directory


In [3]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import os
import sys
import os
from google.colab import drive

# force remount
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)



Drive not mounted, so nothing to flush and unmount.


ValueError: mount failed

In [ ]:
!ls "/content/drive/My Drive/CIS 5200 Final Project/code"


In [6]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/usr/local/lib/python3.12/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.12/dist-packages/IPython/extensions',
 '/root/.ipython']

In [8]:
import sys

# 2. Add your project path to Python's system path
# This tells Python: "Look in this folder when I try to import things"
project_path = '/content/drive/My Drive/CIS 5200 Final Project/code'

if project_path not in sys.path:
    sys.path.append(project_path)

# can use importlib to safely reload if you change the script
import rf_classification
import importlib
importlib.reload(rf_classification)
from rf_classification import train_classifier, InferencePipeline, get_classifier_pipeline


print("Successfully loaded pipeline!")



Successfully loaded pipeline!


In [4]:
# -*- coding: utf-8 -*-
"""rf_pipeline.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1d8BpmDV7Ky474h1FRWA4plu0TTjODC9O
"""

import pandas as pd
import numpy as np
import os
import joblib
import gc
import json
import time
from datetime import datetime

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, roc_auc_score, log_loss, average_precision_score
)

# --- Configuration ---
CLASS_TARGET = 'DEP_DEL15'
LEAKAGE_COLS = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
                'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']

# Define Categoricals (Same as before)
CATEGORICAL_COLS = [
    # Core IDs
    'CARRIER_NAME',
    'DEPARTING_AIRPORT',
    'PREVIOUS_AIRPORT',
    'DESTINATION_AIRPORT',
    # Engineered Routes
    'ROUTE_NAME',      # Origin -> Dest
    'INCOMING_ROUTE',  # Prev -> Origin
    'CARRIER_AIRPORT', # Hub Effect
    # Time
    'DEP_TIME_BLK',
    'MONTH',
    'DAY_OF_WEEK',
    'SEASON',
    # Groups
    'DISTANCE_GROUP',
    'SEGMENT_NUMBER',
    # Weather Flags / binary flags
    'IS_HEAVY_RAIN', 'IS_SNOWY',
    'IS_FREEZING', 'IS_EXTREME_HEAT',
    'AWND_missing', 'TMIN_missing', 'TMAX_missing',
]
# Add WT flags
CATEGORICAL_COLS.extend([f'WT{str(i).zfill(2)}' for i in range(1, 12)])


# --- Transformers ---
class ColumnDropper(BaseEstimator, TransformerMixin):
    """Drop known leakage columns."""
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(
            columns=[c for c in self.columns_to_drop if c in X.columns],
            errors="ignore"
        )


def get_classifier_pipeline():
    """
    Returns a sklearn Pipeline that:
      1. Drops leakage columns
      2. Ordinal-encodes categorical columns
      3. Passes all remaining columns through as numeric features
    """
    preprocessor = ColumnTransformer(
        transformers=[
            (
                "cat",
                OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
                CATEGORICAL_COLS
            )
        ],
        remainder="passthrough"  # keep non-categorical columns as-is
    )

    pipeline = Pipeline(steps=[
        ("dropper", ColumnDropper(LEAKAGE_COLS)),
        ("preprocessor", preprocessor),
    ])

    return pipeline


class InferencePipeline:
    """
    Simple wrapper to keep preprocessor + model together
    and expose predict / predict_proba.
    """
    def __init__(self, preprocessor, model):
        self.preprocessor = preprocessor
        self.model = model

    def predict(self, X):
        X_trans = self.preprocessor.transform(X)
        return self.model.predict(X_trans)

    def predict_proba(self, X):
        X_trans = self.preprocessor.transform(X)
        return self.model.predict_proba(X_trans)

# best_model_params = {'n_estimators': 200, 'n_bins': 128, 'min_samples_split': 5, 'max_features': 'log2', 'max_depth': 15}


# --- Training Function ---
def train_classifier(
    X_train_path,
    y_train_path,
    X_test_path,
    y_test_path,
    description="RF_Classifier",
    checkpoint_dir=None,
    **model_params
):
    """
    Train a RandomForestClassifier with the same structure as your old LGBM pipeline.
    - Loads CSVs
    - Builds preprocessing pipeline
    - Trains RF with stratified validation split
    - Evaluates on test set
    - Saves InferencePipeline + metrics JSON (if checkpoint_dir is provided)
    """

    print(f"\n--- Starting Classifier Training: {description} ---")

    # 1. Load Data
    print("  Loading Training Data...")
    X_train = pd.read_csv(X_train_path)
    y_train = pd.read_csv(y_train_path)[CLASS_TARGET].squeeze()

    # 2. Stratified Split for Train and Validation Sets
    print("  Creating Stratified Validation Split (15%)...")
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.15,
        random_state=42,
        stratify=y_train,
    )
    del X_train, y_train
    gc.collect()

    # 3. Build Pipeline (drop leakage + ordinal encoding)
    pipeline = get_classifier_pipeline()

    # 4. Build Random Forest model
    rf_default_params = {
        "n_estimators": 200,
        "max_depth": 15,
        "max_features": "log2",
        "min_samples_split": 5,
        "min_samples_leaf": 5,
        "n_jobs": -1,
        "class_weight": "balanced",
        "random_state": 42,
    }
    rf_default_params.update(model_params)
    model = RandomForestClassifier(**rf_default_params)

    # 5. Preprocess & Fit
    print("  Preprocessing & Fitting RandomForest...")
    start = time.time()
    X_tr_trans = pipeline.fit_transform(X_tr, y_tr)
    X_val_trans = pipeline.transform(X_val)

    model.fit(X_tr_trans, y_tr)
    train_time = time.time() - start
    print(f"  Training finished in {train_time:.1f}s")

    del X_tr, X_val, X_tr_trans, X_val_trans, y_tr, y_val
    gc.collect()

    # 6. Evaluation on Test Set
    print("  Evaluating on Test Set...")
    X_test = pd.read_csv(X_test_path)
    y_test = pd.read_csv(y_test_path)[CLASS_TARGET].squeeze()

    X_test_trans = pipeline.transform(X_test)
    y_pred = model.predict(X_test_trans)
    y_prob = model.predict_proba(X_test_trans)[:, 1]

    # 7. Report Metrics
    results = {
        "description": description,
        "accuracy": accuracy_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_prob),
        "average_precision/auprc": average_precision_score(y_test, y_prob),
        "log_loss": log_loss(y_test, y_prob),
        "f1": f1_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist(),
        "training_time": train_time,
        "parameters": model.get_params(),
    }

    print(f"  AUC: {results['auc']:.4f}, LogLoss: {results['log_loss']:.4f}")
    print(f"  Precision: {results['precision']:.4f}, Recall: {results['recall']:.4f}")

    # 8. Save Metrics and Pipeline
    if checkpoint_dir:
        os.makedirs(checkpoint_dir, exist_ok=True)
        safe_desc = "".join([c if c.isalnum() else "_" for c in description])
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Save full inference pipeline (preprocessing + RF model)
        path = os.path.join(checkpoint_dir, f"RF_CLS_{safe_desc}_{ts}.joblib")
        joblib.dump(InferencePipeline(pipeline, model), path)
        results["checkpoint_path"] = path

        # Save metrics
        metrics_path = path.replace(".joblib", "_metrics.json")
        with open(metrics_path, "w") as f:
            json.dump(results, f, indent=4)

    return results

# Load and Prep Data (as necessary)

In [9]:
# i=1
# mod = f'{i}'
mod = "added_delay_20251202_034422"

DATA_FILE_PATH = "/content/drive/My Drive/CIS 5200 Final Project/train_test_datasets/"
checkpoint_dir = "/content/drive/My Drive/CIS 5200 Final Project/models/random_forests/"

# 1. Load the dataset and drop rows with missing values
X_train_path = DATA_FILE_PATH + f'X_train_{mod}.csv'
y_train_path = DATA_FILE_PATH + f'y_train_{mod}.csv'

X_test_path = DATA_FILE_PATH + f'X_test_{mod}.csv'
y_test_path = DATA_FILE_PATH + f'y_test_{mod}.csv'

# Regression
REGR_TARGET = 'DEP_ADDED_DELAY'

# Classification
CLASS_TARGET='DEP_DEL15'

## HyperParameter Randomized Search, cuML random forests

In [ ]:
target = 'DEP_ADDED_DELAY'
# 1. Load a SUBSET of data for tuning (e.g., 100k rows)
# Tuning on the full 1.2M dataset is overkill and slow for CV
X_tune = pd.read_csv(X_train_path, nrows=200000)
y_tune = pd.read_csv(y_train_path, nrows=200000)[target]

# 2. Get your existing preprocessing pipeline (without the model)
# note: flight_delay_pipeline.py returns a pipeline ending in 'preprocessor'
prep_pipeline = get_preprocessing_pipeline()

# 3. Create the Full Pipeline (Prep + GPU Model)
# We add the model as the final step
full_pipeline = Pipeline(steps=[
    ('prep', prep_pipeline),
    ('model', cuRF_Regressor(verbose=0)) # Turn off model verbosity to keep output clean
])

# 4. Define the Parameter Grid
# Note the 'model__' prefix which tells sklearn to apply these to the 'model' step
param_dist = {
    'model__n_estimators': [100, 150, 200, 250, 300],     # More trees = better but slower
    'model__max_depth': [10, 15, 20, 25, 30],       # Depth controls overfitting
    'model__max_features': ['sqrt', 'log2'],    # Features per split
    'model__n_bins': [32, 64, 128, 256],             # Lower bins = Faster, less GPU RAM
    'model__min_samples_split': [2, 5, 10],
}

# 5. Setup RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_dist,
    n_iter=20,          # Try 20 random combinations
    cv=3,               # 3-Fold CV is enough for tuning
    scoring='neg_mean_absolute_error',
    n_jobs=1,           # CRITICAL: Keep this at 1 to avoid GPU OOM
    verbose=3,          # See progress
    random_state=42
)

# 6. Run the search
print("Starting Hyperparameter Sweep...")
search.fit(X_tune, y_tune)

print(f"Best Params: {search.best_params_}")
# The "best_score_" will now be negative MAE (e.g., -15.4)
print(f"Best MAE (Validation): {search.best_score_}")

# 7. (Optional) Save the best params to use in your main script
best_model_params = {k.replace('model__', ''): v for k, v in search.best_params_.items()}
print("Params ready for train_model:", best_model_params)

In [ ]:

print('Best Parameters for RF with Randomized Grid Search')
pprint(best_model_params)

Best Parameters for RF with Randomized Grid Search


NameError: name 'pprint' is not defined

# Setup the Random Forest
Some important hyperparameters for a Random Forest:
1. n_estimators: the # of decision trees to ensemble (default=100)

2. max_features: the max # of features a decision tree will be allowed to consider (default = sqrt(N_features))

3. max_depth: max depth a tree will be allowed to reach (default = inf)

4. max_leaf_nodes: max number of leaf nodes a tree can hold (default = inf)

5. max_sample: how many samples from the training set that a decision tree can see (default = entire training set)

6. min_samples_split: The threshold for which a node can be split (default = 2)

More hyperparams [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
from pprint import pprint

In [ ]:
# define hyperparameters for RandomForestRegressor
# n_estimators=5000
# max_features="sqrt"
# max_depth=16 # cuML default is 16, sklearn default is inf
# max_leaves=-1 #inf
# max_samples=1.0 # all samples
# min_samples_split=2
# criterion="squared_error" # "absolute error" # is an option, but much slower per the sklearn docs
# verbose=2
# n_binsint (default = 128) # Maximum number of bins used by the split algorithm per feature.
# For large problems, particularly those with highly-skewed input data, increasing the number of bins may improve accuracy.
# https://docs.rapids.ai/api/cuml/stable/api/#output-data-type-configuration, default is 'input', so for pandas inputs == numpy array outputs
best_model_params = {'n_estimators': 200, 'n_bins': 128, 'min_samples_split': 5, 'max_features': 'log2', 'max_depth': 15}


## Regression
# params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
#           'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
#           'n_bins': 256, 'criterion': 'squared_error', 'verbose': 2}

# params.update(best_model_params)
# pprint(params)
# model=cuRF_Regressor(**params)

## Classification
params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
          'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
          'n_bins': 256, 'criterion': 'entropy', 'verbose': 2}

params.update(best_model_params)
model=cuRF_Classifier(**params)

In [ ]:
pprint(X_train_path)
pprint(X_test_path)
pprint(y_train_path)
pprint(y_test_path)

('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/X_train_added_delay_20251202_034422.csv')
('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/X_test_added_delay_20251202_034422.csv')
('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/y_train_added_delay_20251202_034422.csv')
('/content/drive/My Drive/CIS 5200 Final '
 'Project/train_test_datasets/y_test_added_delay_20251202_034422.csv')


# Train Model on Datasets

## Full Dataset

In [10]:
# i=1

model_type = 'classification'
description = f"cuML RFClass Dataset {mod}"

# model_type = 'regression'
# description = f"cuML RFReg Dataset {mod}"

# DATA_FILE_PATH = "/content/drive/My Drive/CIS 5200 Final Project/train_test_datasets/no_encodings/"
# checkpoint_dir = "/content/drive/My Drive/CIS 5200 Final Project/models/random forests/"

# # 1. Load the dataset and drop rows with missing values
# X_train_path = DATA_FILE_PATH + f'X_train_{mod}_no_encoding.csv'
# y_train_path = DATA_FILE_PATH + f'y_train_{mod}_no_encoding.csv'

# X_test_path = DATA_FILE_PATH + f'X_test_{mod}_no_encoding.csv'
# y_test_path = DATA_FILE_PATH + f'y_test_{mod}_no_encoding.csv'



# --- Training Function ---
# def train_classifier(
#     X_train_path,
#     y_train_path,
#     X_test_path,
#     y_test_path,
#     description="RF_Classifier",
#     checkpoint_dir=None,
#     **model_params
# ):

results = train_classifier(X_train_path=X_train_path, y_train_path=y_train_path,
                      X_test_path=X_test_path, y_test_path=y_test_path, description=description,
                      checkpoint_dir=checkpoint_dir)


--- Starting Classifier Training: cuML RFClass Dataset added_delay_20251202_034422 ---
  Loading Training Data...
  Creating Stratified Validation Split (15%)...
  Preprocessing & Fitting RandomForest...
[cuml.accel] `RandomForestClassifier.fit` falling back to CPU: `class_weight` is not supported
[cuml.accel] `RandomForestClassifier.fit` ran on CPU
  Training finished in 2179.7s
  Evaluating on Test Set...
[cuml.accel] `RandomForestClassifier.predict` ran on CPU
[cuml.accel] `RandomForestClassifier.predict_proba` ran on CPU
  AUC: 0.8407, LogLoss: 0.5277
  Precision: 0.3971, Recall: 0.7724


# Run a Dummy Inference Example

In [ ]:
print(results)

{'description': 'cuML RFClass Dataset full', 'training_time_sec': 192.36317372322083, 'timestamp': '2025-11-29 22:07:08', 'pipeline': <flight_delay_pipeline.InferencePipeline object at 0x7b0c9aa0b8f0>, 'accuracy': 0.8314834271262256, 'f1': 0.0637138600147684, 'precision': 0.7586888657648283, 'recall': 0.03325321426453887, 'confusion_matrix': array([[1050020,    2319],
       [ 211966,    7291]]), 'checkpoint_path': '/content/drive/My Drive/CIS 5200 Final Project/models/random forests/cuML_RFClass_Dataset_full_20251129_220708.joblib'}


In [ ]:
results.keys()

dict_keys(['description', 'training_time_sec', 'timestamp', 'pipeline', 'accuracy', 'f1', 'precision', 'recall', 'confusion_matrix', 'checkpoint_path'])

In [ ]:
from joblib import load

rf_performance = load(results["checkpoint_path"])

rf_performance

{'description': 'cuML RFClass Dataset added_delay_20251202_034422',
 'training_time_sec': 168.03658390045166,
 'timestamp': '2025-12-04 05:11:46',
 'pipeline': <flight_delay_pipeline.InferencePipeline at 0x783096220680>,
 'accuracy': 0.8293420237245163,
 'f1': 0.02556779912169626,
 'precision': 0.8264150943396227,
 'recall': 0.012984762174069698,
 'confusion_matrix': array([[1051741,     598],
        [ 216410,    2847]])}

In [ ]:
X_tune = pd.read_csv(X_train_path, nrows=200000)
y_tune = pd.read_csv(y_train_path, nrows=200000)[CLASS_TARGET]

In [ ]:
pprint(X_tune[:1])
pprint(y_tune[:1])

   MONTH  DAY_OF_WEEK DEP_TIME_BLK  DISTANCE_GROUP  SEGMENT_NUMBER  \
0      5            7    1000-1059               4               1   

   CONCURRENT_FLIGHTS  NUMBER_OF_SEATS            CARRIER_NAME  \
0                   6              140  American Airlines Inc.   

   AIRPORT_FLIGHTS_MONTH  AIRLINE_FLIGHTS_MONTH  ...  WT05  WT06  WT07  WT08  \
0                   2588                  78894  ...   0.0   0.0   0.0   0.0   

   WT09  WT10 WT11 AWND_missing  TMIN_missing  TMAX_missing  
0   0.0   0.0  0.0        False         False         False  

[1 rows x 38 columns]
0    0
Name: DEP_DEL15, dtype: int64


In [ ]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# # 1. Define the wrapper class (or import it from your pipeline script)
# class InferencePipeline:
#     def __init__(self, preprocessor, model):
#         self.preprocessor = preprocessor
#         self.model = model
#     def predict(self, X):
#         X_trans = self.preprocessor.transform(X).astype(np.float32)
#         return self.model.predict(X_trans)

# 2. Load the checkpoint
checkpoint_dir = '/content/drive/My Drive/CIS 5200 Final Project/models/random_forests/'
chk_path = "/content/drive/My Drive/CIS 5200 Final Project/models/random_forests/cuML_RFClass_Dataset_added_delay_20251202_034422_20251204_051146.joblib"
# chk_path = checkpoint_dir + 'cuML_RFClass_Dataset_full_20251128_220339.joblib'
checkpoint = joblib.load(chk_path)

# 3. Extract your custom pipeline from the results dictionary
inference_pipe = checkpoint['pipeline']

model = inference_pipe.model

In [ ]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

AttributeError: 'RandomForestClassifier' object has no attribute 'feature_importances_'

In [ ]:

# 5. Convert to DataFrame and Predict
df_example = X_tune[:1]


# Run Inference
prediction = inference_pipe.predict(df_example)
# Regression
# print(f"Predicted Delay: {prediction[0]:.2f} minutes")
# Clasisfication
print(f"Predicted Class (0 = Ontime, 1 = Delayed >= 15): {prediction[0]}")

In [ ]:
# best_model_params = {'n_estimators': 200, 'n_bins': 128, 'min_samples_split': 5, 'max_features': 'log2', 'max_depth': 15}


# Regression
params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
          'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
          'n_bins': 256, 'criterion': 'squared_error', 'verbose': 2}

params.update(best_model_params)
pprint(params)
model=cuRF_Regressor(**params)

## Classification
# params = {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30,
#           'max_leaves': -1, 'max_samples': 1.0, 'min_samples_split': 2,
#           'n_bins': 256, 'criterion': 'entropy', 'verbose': 2}

# params.update(best_model_params)
# pprint(params)
# model=cuRF_Classifier(**params)

{'criterion': 'squared_error',
 'max_depth': 15,
 'max_features': 'log2',
 'max_leaves': -1,
 'max_samples': 1.0,
 'min_samples_split': 5,
 'n_bins': 128,
 'n_estimators': 200,
 'verbose': 2}


In [ ]:
mask_delay = y_train["DEP_ADDED_DELAY"] > 0

X_train_reg = X_train.loc[mask_delay].copy()
y_train_reg = y_train.loc[mask_delay].copy()


In [ ]:


results_regressor = train_model(X_train_path=X_train_path, y_train_path=y_train_path,
                      X_test_path=X_test_path, y_test_path=y_test_path, model=model,
                      checkpoint_dir=checkpoint_dir, description=description,
                      model_type=model_type, model_name='rapid_rf')

In [ ]:
# import pandas as pd
# import numpy as np
# import os
# import joblib
# import gc
# import json
# import time
# from datetime import datetime

# from sklearn.pipeline import Pipeline
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.model_selection import train_test_split
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import (
#     root_mean_squared_error, mean_absolute_error, r2_score,
#     mean_tweedie_deviance, mean_gamma_deviance, mean_absolute_percentage_error
# )

# # --- Configuration ---
# REGR_TARGET = 'DEP_ADDED_DELAY'
# LEAKAGE_COLS = ['CARRIER_DELAY', 'WEATHER_DELAY',
#                 'NAS_DELAY', 'SECURITY_DELAY',
#                 'LATE_AIRCRAFT_DELAY']

# CATEGORICAL_COLS = [
#     'CARRIER_NAME', 'DEPARTING_AIRPORT', 'PREVIOUS_AIRPORT', 'DESTINATION_AIRPORT',
#     'ROUTE_NAME', 'INCOMING_ROUTE', 'CARRIER_AIRPORT',
#     'DEP_TIME_BLK', 'MONTH', 'DAY_OF_WEEK', 'SEASON',
#     'DISTANCE_GROUP', 'SEGMENT_NUMBER',
#     'IS_HEAVY_RAIN', 'IS_SNOWY', 'IS_FREEZING', 'IS_EXTREME_HEAT',
#     'AWND_missing', 'TMIN_missing', 'TMAX_missing',
# ]
# CATEGORICAL_COLS.extend([f'WT{str(i).zfill(2)}' for i in range(1, 12)])


# # --- Transformers ---
# class ColumnDropper(BaseEstimator, TransformerMixin):
#     def __init__(self, columns_to_drop):
#         self.columns_to_drop = columns_to_drop

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         return X.drop(columns=[c for c in self.columns_to_drop if c in X.columns])


# def get_regressor_pipeline():
#     """
#     Drop leakage columns, then ordinal-encode categorical features and
#     passthrough the rest.
#     """
#     preprocessor = ColumnTransformer(
#         transformers=[
#             (
#                 "cat",
#                 OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
#                 CATEGORICAL_COLS,
#             ),
#         ],
#         remainder="passthrough",  # keep non-categorical columns as-is
#     )

#     pipe = Pipeline(steps=[
#         ('dropper', ColumnDropper(LEAKAGE_COLS)),
#         ('preprocessor', preprocessor),
#     ])
#     return pipe


# class InferencePipeline:
#     """
#     Wrapper that keeps preprocessor + model together and exposes predict.
#     """
#     def __init__(self, preprocessor, model):
#         self.preprocessor = preprocessor
#         self.model = model

#     def predict(self, X):
#         X_trans = self.preprocessor.transform(X)
#         return self.model.predict(X_trans)


# # --- Training Function ---
# def train_regressor(
#     X_train_path,
#     y_train_path,
#     X_test_path,
#     y_test_path,
#     stopping_rounds=100,  # kept for API compatibility, not used by RF
#     description="RF_Regressor",
#     pos_only=False,
#     extra_metrics=None,
#     checkpoint_dir=None,
#     **model_params,
# ):
#     """
#     Train a RandomForestRegressor with your flight-delay features.

#     - Optionally trains only on rows with DEP_ADDED_DELAY > 0 (pos_only=True)
#     - Uses a 15% validation split (for monitoring only)
#     - Saves an InferencePipeline (preprocessor + RF model) plus metrics JSON
#     """

#     print(f"\n--- Starting Regressor Training: {description} ---")

#     # 1. Load Data
#     print("  Loading Training Data...")
#     X_train = pd.read_csv(X_train_path)
#     y_train = pd.read_csv(y_train_path)[REGR_TARGET].squeeze()

#     if pos_only:
#         print(f"  Filtering for positive delays... (Original: {len(X_train)})")
#         mask_pos = (y_train > 0)
#         X_train = X_train[mask_pos]
#         y_train = y_train[mask_pos]
#         print(f"  Training samples remaining: {len(X_train)}")
#     else:
#         print("  pos_only is False, training on WHOLE training set")

#     # 2. Split
#     print("  Creating Validation Split (15%)...")
#     X_tr, X_val, y_tr, y_val = train_test_split(
#         X_train,
#         y_train,
#         test_size=0.15,
#         random_state=42,
#     )
#     del X_train, y_train
#     gc.collect()

#     # 3. Pipeline Setup
#     pipeline = get_regressor_pipeline()

#     # 4. Build RandomForestRegressor
#     rf_params = {
#         "n_estimators": 300,
#         "max_depth": None,
#         "max_features": "sqrt",
#         "min_samples_split": 2,
#         "min_samples_leaf": 1,
#         "n_jobs": -1,
#         "random_state": 42,
#     }
#     rf_params.update(model_params)
#     model = RandomForestRegressor(**rf_params)

#     print("  Preprocessing & Fitting RandomForestRegressor...")
#     X_tr_trans = pipeline.fit_transform(X_tr, y_tr)
#     X_val_trans = pipeline.transform(X_val)

#     start = time.time()
#     model.fit(X_tr_trans, y_tr)
#     train_time = time.time() - start
#     print(f"  Training finished in {train_time:.1f}s")

#     del X_tr, X_val, X_tr_trans, X_val_trans, y_tr, y_val
#     gc.collect()

#     # --- Save model first ---
#     results = {
#         "description": description,
#         "training_time": train_time,
#         "parameters": model.get_params(),
#     }

#     if checkpoint_dir:
#         os.makedirs(checkpoint_dir, exist_ok=True)
#         safe_desc = "".join([c if c.isalnum() else "_" for c in description])
#         ts = datetime.now().strftime("%Y%m%d_%H%M%S")

#         path = os.path.join(checkpoint_dir, f"{safe_desc}_{ts}.joblib")
#         print(f"  Saving model to: {path} ...")
#         joblib.dump(InferencePipeline(pipeline, model), path)
#         results["checkpoint_path"] = path

#     # 5. Evaluate
#     print("  Evaluating on Test set examples...")
#     try:
#         X_test = pd.read_csv(X_test_path)
#         y_test = pd.read_csv(y_test_path)[REGR_TARGET].squeeze()

#         if pos_only:
#             print("  pos_only is True, testing on only positive examples")
#             mask_test_pos = (y_test > 0)
#             X_test = X_test[mask_test_pos]
#             y_test = y_test[mask_test_pos]
#         else:
#             print("  pos_only is False, testing on WHOLE test set")

#         X_test_trans = pipeline.transform(X_test)
#         y_pred = model.predict(X_test_trans)

#         y_pred_safe = np.maximum(y_pred, 1e-9)

#         results.update({
#             "rmse": root_mean_squared_error(y_test, y_pred),
#             "mae": mean_absolute_error(y_test, y_pred),
#             "mape": mean_absolute_percentage_error(y_test, y_pred),
#             "r2": r2_score(y_test, y_pred),
#         })

#         # Advanced metrics
#         try:
#             results["gamma_deviance"] = mean_gamma_deviance(y_test, y_pred_safe)
#         except Exception as e:
#             print(f"  Warning: Gamma deviance calc failed: {e}")

#         try:
#             results["tweedie_deviance"] = mean_tweedie_deviance(
#                 y_test, y_pred_safe, power=1.5
#             )
#         except Exception as e:
#             print(f"  Warning: Tweedie deviance calc failed: {e}")

#         # Extra custom metrics
#         if extra_metrics:
#             for name, func in extra_metrics.items():
#                 try:
#                     results[name] = func(y_test, y_pred)
#                 except Exception as e:
#                     print(f"  Warning: Custom metric '{name}' failed: {e}")

#         print(f"  RMSE:  {results.get('rmse', 'N/A'):.4f}")
#         print(f"  MAE:   {results.get('mae', 'N/A'):.4f}")
#         print(f"  MAPE:  {results.get('mape', 'N/A'):.4%}")

#         # Save Metrics JSON
#         if checkpoint_dir and "checkpoint_path" in results:
#             json_path = results["checkpoint_path"].replace(".joblib", "_metrics.json")
#             with open(json_path, "w") as f:
#                 json.dump(results, f, indent=4)
#             print(f"  Metrics saved to: {json_path}")

#     except Exception as e:
#         print("\n!!! CRITICAL WARNING: Evaluation crashed, but Model IS SAVED !!!")
#         print(f"Error details: {e}")

#     return results


In [ ]:
results_reg = train_regressor(
    X_train_path,
    y_train_path,
    X_test_path,
    y_test_path,
    description="RF_Reg_pos_only",
    pos_only=True,
    checkpoint_dir=checkpoint_dir
)


--- Starting Regressor Training: RF_Reg_pos_only ---
  Loading Training Data...
  Filtering for positive delays... (Original: 5086383)
  Training samples remaining: 877028
  Creating Validation Split (15%)...
  Preprocessing & Fitting RandomForestRegressor...
[cuml.accel] `RandomForestRegressor.fit` ran on GPU
  Training finished in 35.8s
  Saving model to: /content/drive/My Drive/CIS 5200 Final Project/models/random_forests/RF_Reg_pos_only_20251204_165715.joblib ...
[cuml.accel] `RandomForestRegressor` fitted attributes synced to CPU
  Evaluating on Test set examples...
  pos_only is True, testing on only positive examples
[cuml.accel] `RandomForestRegressor.predict` ran on GPU
  RMSE:  22.0088
  MAE:   17.5426
  MAPE:  238.2439%
  Metrics saved to: /content/drive/My Drive/CIS 5200 Final Project/models/random_forests/RF_Reg_pos_only_20251204_165715_metrics.json
